# Overview
In this file, the MNI space mask image is shrunk, so that the spheres with all voxels within the brain (original mask) are used for classification. In the naive space calculation, similar strategy has been implemented but to assure the spheres are complete (not touching the surface of brain imaging (ie 0, or 64). 

    

In [1]:
import numpy as np
import pandas as pd
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
from scipy.ndimage import binary_erosion
import itertools
import multiprocessing
from multiprocessing import Pool
from importlib import reload #import function "reload"
import sys

import warnings
warnings.filterwarnings("ignore")
import os
import glob
from glob import glob
from scipy.io import loadmat
import math
import timeit
from timeit import default_timer as timer

plt.style.use('classic')
%matplotlib inline

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
#from sklearn.svm import LinearSVC

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupKFold
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier


In [ ]:
from MVPA_func import 

### Load MNI space mask

In [4]:
MASK_Original_fileName='C://Users/user/Documents/MATLAB/spm12/tpm/mask_ICV.nii'
MASK_Original_img=nib.load(MASK_Original_fileName)
MASK_Original_data=MASK_Original_img.get_data()
voxel_dims = MASK_Original_data.shape
print(voxel_dims)

(121, 145, 121)


In [20]:
radius=0
MVPAFile='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/SpotlightR'+str(radius)+'/MVPA_AB_04152014.nii.gz'
subj_img=nib.load(MVPAFile)
VoxelSizeNaive=subj_img.header.get_zooms()[0]

In [39]:
for radius in range(1,5):
    # Load the mask brain image
    mask_img = nib.load(MASK_Original_fileName)
    mask_data = mask_img.get_fdata()

    # Define the distance (in voxels) from the surface to keep
    VoxelSizeMNI=MASK_Original_img.header.get_zooms()[0]
    shrinkSize=math.ceil(radius*VoxelSizeNaive/VoxelSizeMNI)
    print(f'shrink Size is for Naive space radius of {radius} is {shrinkSize}')
    # Apply binary erosion to shrink the mask
    shrunken_mask = binary_erosion(mask_data, iterations=shrinkSize)


    # Save the modified mask image if needed
    modified_mask_img = nib.Nifti1Image(shrunken_mask.astype(np.uint8), mask_img.affine)
    nib.save(modified_mask_img, 'C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/mask_ICV_R'+str(radius)+'.nii.gz')

shrink Size is for Naive space radius of 1 is 3
shrink Size is for Naive space radius of 2 is 6
shrink Size is for Naive space radius of 3 is 8
shrink Size is for Naive space radius of 4 is 11


4

In [22]:
VoxelSizeMNI

1.5

In [5]:
print(MASK_Original_img.header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [  3 121 145 121   1   1   1   1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [-1.   1.5  1.5  1.5  0.   0.   0.   0. ]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b'Intracranial region'
aux_file        : b''
qform_code      : mni
sform_code      : mni
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [-1.5

In [6]:
MASK_Original_img.header

### Load SPM.mat to extract experiment info
This cell loads spm.mat file for each participant and from the data, extracts the number of runs, the path for each stat file, etc

In [5]:
# NEW note: I need to change neg to sad & Fear, 3 December 2023
# note: I don't know why the last item is repeated! but it doesn't make a misatke

conditions=['Neu','Neg','Pos','Targ']
ZstatInfo=pd.DataFrame(columns=['Subj','Run','Folder']+conditions)
for subj in subj_list:
    #print('Participant: '+subj)
    # Zstat files and their condisitons are in SPM filesWe repeat loading in the dataframe to avoid dividing the onsets by 2 multiple times ...
    SubjSPMFolder='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/'+subj+'/3cond_dist2/'
    os.path.isdir(SubjSPMFolder)
    myZstat=[]
    
    nRuns = len(os.listdir(SubjSPMFolder))
    iRuns = 0
    for myRun in os.listdir(SubjSPMFolder):
        ZstatInfotmp=pd.DataFrame(columns=ZstatInfo.columns,index=[0])
        ZstatInfotmp['Subj'][0]=subj
        
        RunSubjSPM=SubjSPMFolder+myRun+'/SPM.mat'
        ZstatInfotmp['Folder'][0]=SubjSPMFolder+myRun+'/'

        #print('   loading SPM.mat for '+myRun+'...')
        mat = loadmat(RunSubjSPM)
        mdata=mat['SPM'][0,0]
        ndata=mdata['Sess'][0,0]
        odata=ndata['U']
        pdata=odata['name'][0]
        ZstatNames=[pdata[i][0,0][0] for i in range(len(pdata))]
        myZstat.append(ZstatNames)

        ZstatInfotmp['Run'][0]=iRuns+1
        old_ind=len(myZstat[0])+1
        for iCond in reversed(conditions):    
            Start_ind=myZstat[0].index(iCond+'_1')+1
            #print(Start_ind)
            ZstatInfotmp[iCond][0]=['spmT_'+str(val).zfill(4)+'.nii' for val in list(range(Start_ind, old_ind))]#list(range(Start_ind, old_ind))
            old_ind=Start_ind
        ZstatInfo=ZstatInfo.append(ZstatInfotmp,ignore_index=True)
        iRuns = iRuns+1
 

### find Voxel dimension by openning one of the image files


In [6]:
MyZstat=ZstatInfo.loc[(ZstatInfo['Subj']==subj) & (ZstatInfo['Run']==1)]
MyZstat.reset_index(inplace = True, drop = True)
MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
#print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)


(64, 64, 28)


In [19]:
# SpherePoints(np.array([1,1,0]) , radius, voxel_dims)
IncompleteSpherePoints(np.array([1,1,0]) , radius, voxel_dims)
    

array([[[-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          3]],

       [[ 1,  0,  0,  0,  1,  1,  1,  2,  2,  2, -1,  0,  0,  0,  1,  1,
          1,  1,  1,  2,  2,  2,  3,  0,  0,  0,  1,  1,  1,  2,  2,  2,
          1]],

       [[ 0, -1,  0,  1, -1,  0,  1, -1,  0,  1,  0, -1,  0,  1, -2, -1,
          0,  1,  2, -1,  0,  1,  0, -1,  0,  1, -1,  0,  1, -1,  0,  1,
          0]]])

In [11]:
# Calculate the spotlights voxels for all points and keep them in AllPoints 
start = timer()

radius = 2 #spheres  radius 
tmpPoints = SpherePoints(np.array([26,26,16]) , radius, voxel_dims) #generate a random sphere to find its dimension
print('size of shperes:',tmpPoints.shape)
AllZeroSphere = np.zeros((1,tmpPoints.shape[2]))
AllZeroSpherePoints=np.zeros(tmpPoints.shape)
cnt=0
for MyCenter in itertools.product(range(0,voxel_dims[0]), range(0,voxel_dims[1]), range(0,voxel_dims[2])):
    #print(MyCenter)
    myPoints = IncompleteSpherePoints(np.array(MyCenter) , radius, voxel_dims)
    if myPoints.shape==(3,1,0): myPoints=AllZeroSpherePoints
    if cnt==0:
        AllPoints = np.array(myPoints,ndmin=4)
        cnt=1
    else:
        AllPoints = np.concatenate((AllPoints,np.array(myPoints,ndmin=4)),axis=0)
#     if myPoints.shape!=(3,1,0): sphereVal = data[myPoints[0],myPoints[1],myPoints[2]]
#     else: sphereVal = AllZeroSphere
        
#    if myPoints.shape!=(0,): sphereVal = SphereValue(myPoints,data)
#     if sphereVal.shape!=(0,): sphereValOut=


end = timer()
print(end-start)
np.save('Radius'+str(radius)+'_points.npy',AllPoints)
# aa=np.load('Radius'+str(radius)+'_points.npy',AllPoints)

size of shperes: (3, 1, 33)
1520.4460888999747


# This code does classification based on spot light search, serial processing, slow. Don't run it
start = timer()
CLconditions=['Neu','Neg','Targ']
radius = 1 #spheres radius


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

subj_list_one=[subj_list[1]]; 
Nsubj=len(subj_list_one)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list_one):
    
    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    
    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    for iFeature in range(AllPoints.shape[0]): #iFeature=40836, range(56500,56610):

        SpotLight_performance[iFeature]= LoopiFeature(iFeature, AllPoints,ZstatInfoSubj ,CLconditions, Alldata, pipeSpotLight)
        
end = timer()
print(end-start)
#SpotLight_performance[56500:56610]

array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0.]]])

In [ ]:
# This code does classification based on spot light search, parallel processing, fast, 
# for complete & incomplete spheres, and limited to Brain Extracted Image

CLconditions=['Neu','Neg','Targ']


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

# subj_list_one=[subj_list[1]]; 
# Nsubj=len(subj_list_one)
Nsubj=len(subj_list)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list):
    print(subj)
    start = timer()

    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
    Zstattmp.reset_index(inplace = True, drop = True)
    MyFiletmp=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0]
    
    myimg_affine=nib.load(MyFiletmp).affine

    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
    aa1= range(AllPoints.shape[0])
    aa2=[DataArgDict]*len(aa1)
    aa=[*zip(aa1, aa2)]

    if __name__ == "__main__":

        pool = Pool(multiprocessing.cpu_count()-2)
        resultsN = pool.map(multi_run_wrapperLoopiFeatureDict,aa)

        resultsN=np.array(resultsN).reshape(voxel_dims)

        nft_img = nib.Nifti1Image(resultsN, myimg_affine)
        OutFile='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/SpotlightR'+str(radius)+'/MVPA_'+subj+'.nii.gz'
        nib.save(nft_img, OutFile)

    end = timer()
    print(end-start)
# print(resultsN[56500:56510])

In [ ]:
# This code does classification based on spot light search, parallel processing, fast
CLconditions=['Neu','Neg','Targ']


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

# subj_list_one=[subj_list[1]]; 
# Nsubj=len(subj_list_one)
Nsubj=len(subj_list)
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate(subj_list):
    print(subj)
    start = timer()

    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
    Zstattmp.reset_index(inplace = True, drop = True)
    MyFiletmp=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0]
    
    myimg_affine=nib.load(MyFiletmp).affine

    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
    aa1= range(AllPoints.shape[0])
    aa2=[DataArgDict]*len(aa1)
    aa=[*zip(aa1, aa2)]

    if __name__ == "__main__":

        pool = Pool(multiprocessing.cpu_count()-2)
        resultsN = pool.map(multi_run_wrapperLoopiFeatureDict,aa)

        resultsN=np.array(resultsN).reshape(voxel_dims)

        nft_img = nib.Nifti1Image(resultsN, myimg_affine)
        MVPAFile='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/SpotlightR'+str(radius)+'/MVPA_AB_04152014.nii.gz'
        nib.save(nft_img, OutFile)

    end = timer()
    print(end-start)
# print(resultsN[56500:56510])

AB_04152014


In [13]:
Nsubj

11

In [ ]:
# This code does classification based on spot light search, serial processing (for trouble shooting)
CLconditions=['Neu','Neg','Targ']


##LinearSVC(random_state=0))
clf = SVC(kernel='linear',probability=True)
pipeSpotLight = Pipeline([('scaler', StandardScaler()), ('clf', clf)])

# subj_list_one=[subj_list[1]]; 
# Nsubj=len(subj_list_one)
Nsubj=len([subj_list[6]])
all_performance = np.zeros(Nsubj)
for i_sub, subj in enumerate([subj_list[6]]):
    print(subj)
    start = timer()

    ZstatInfoSubj=ZstatInfo.loc[ZstatInfo['Subj']==subj]
    Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
    Zstattmp.reset_index(inplace = True, drop = True)
    MyFiletmp=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0]
    
    myimg_affine=nib.load(MyFiletmp).affine

    Alldata=LoadAllNii(ZstatInfoSubj,CLconditions) # load Alldata
         
    DataArgDict= {'AllPoints': AllPoints,
              'ZstatInfoSubj':ZstatInfoSubj ,
              'CLconditions': CLconditions, 
              'Alldata': Alldata,
              'pipeSpotLight':pipeSpotLight} 
    SpotLight_performance=np.zeros(AllPoints.shape[0])
    
#     aa1= range(AllPoints.shape[0])
#     aa2=[DataArgDict]*len(aa1)
#     aa=[*zip(aa1, aa2)]

    resultsN = np.zeros(AllPoints.shape[0])
    for iFeature in range(AllPoints.shape[0]): #iFeature=40836, range(56500,56610):

        resultsN[iFeature]= LoopiFeatureDict(iFeature, DataArgDict)
    
    resultsN=np.array(resultsN).reshape(voxel_dims)

    nft_img = nib.Nifti1Image(resultsN, myimg_affine)
    OutFile='C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/MVPA/SpotlightR'+str(radius)+'/MVPA_'+subj+'.nii.gz'
    nib.save(nft_img, OutFile)

    end = timer()
    print(end-start)
# print(resultsN[56500:56510])

ET_12102013


In [41]:
Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
Zstattmp.reset_index(inplace = True, drop = True)
MyFile=Zstattmp['Folder'][0]+Zstattmp['Neg'][0][0]
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)

C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/MRI_data/Analysis/SPM/AM_05082014/3cond_dist2/run01/spmT_0009.nii


In [40]:
Zstattmp=ZstatInfoSubj.loc[ZstatInfoSubj['Run']==1]
Zstattmp.reset_index(inplace = True, drop = True)
Zstattmp

,Subj,Run,Folder,Neu,Neg,Pos,Targ
0,AM_05082014,1,C://Users/user/Documents/Reyhaneh/EEG+fMRI/EO/...,"[spmT_0001.nii, spmT_0002.nii, spmT_0003.nii, ...","[spmT_0009.nii, spmT_0010.nii, spmT_0011.nii, ...",[spmT_0017.nii],"[spmT_0018.nii, spmT_0019.nii, spmT_0020.nii, ..."


In [ ]:

MyFile=MyZstat['Folder'][0]+MyZstat['Neu'][0][0]
#print(MyFile)
data = nib.load(MyFile).get_data()
voxel_dims = data.shape
print(voxel_dims)

In [ ]:
statOut=np.array(resultsN).reshape(voxel_dims)
statOut.shape
nib.save(statOut)

In [ ]:
cnt=0
tmpVal=[]
for tmp in itertools.product(range(0,1), range(0,2), range(0,3)):
    print(f'{tmp}: {cnt}')
    tmpVal.append(cnt)
    cnt+=1
print(tmpVal)
tmpVal=np.array(tmpVal)
tt=tmpVal.reshape(1,2,3)
tt.shape

In [ ]:
tt[0,1,2]

In [ ]:
aa1= range(AllPoints.shape[0])
aa2=[DataArgDict]*len(aa1)
aa=[*zip(aa1, aa2)]

#def multi_run_wrapperLoopiFeatureDict(args): # If I call it here, Jupyter notebook will freeze
#    return LoopiFeatureDict(*args) 
start = timer()

if __name__ == "__main__":
    
    pool = Pool(multiprocessing.cpu_count())
    results = pool.map(multi_run_wrapperLoopiFeatureDict,aa)
    #print(results)
end = timer()
print(end-start)    
print(results[56500:56510])

If it is needed to modify a funciton and relaod it:

In [ ]:
#del multi_run_wrapperLoopiFeatureDict
#from MVPA_func import multi_run_wrapperLoopiFeatureDict
multi_run_wrapperLoopiFeatureDict = reload(sys.modules["MVPA_func"]).multi_run_wrapperLoopiFeatureDict  # reload() returns the new module
LoopiFeatureDict = reload(sys.modules["MVPA_func"]).LoopiFeatureDict  # reload() returns the new module


In [ ]:
SpotLight_performance[56500:56510]